<a href="https://colab.research.google.com/github/maniceet/How-do-you-like-them-Apples/blob/master/How_do_you_like_them_Apples_(Testing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing Packages
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import re, string, unicodedata
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from functools import reduce
from fastai.text import *
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
#Defining class to read txt file by paragraphs rather than by line
class ByParagraph:


    def __init__(self, text):
        self.text = text


    def __iter__(self):
        return self
        par = ""


    def __next__(self):
        par = []
        for line in self.text:
            if line.isspace():
                break
            par.append(line.strip())
        else:
            if not par:
                raise StopIteration()
        return ' '.join(par)

In [0]:
#Cleaning Training Data

#Remove HTML Tags
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text
r_html = lambda x: strip_html_tags(x)

#Remove accented characters

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

r_accented = lambda x: remove_accented_chars(x)

#Expand Contractions

from contraction import expand_contractions
expand = lambda x: expand_contractions(x)


#Remove special characters

def remove_special_characters(text, remove_digits=False):
    #text = text.lower()
    text = re.sub('\w*\d\w*', '', text) #Removing words that have numbers in them
    pattern = r'[^a-zA-z0-9.\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    pattern = r'[''""_]' #Removing punctuations and underscores
    text = re.sub(pattern, '', text)
    text = re.sub(r'(.)\1+', r'\1\1', text)   #Removing characters that appear more than twice
    text = re.sub(r'\[', "",text)
    text = re.sub(r'\]', "",text)
    text = text.replace('\t', ' ')
    return text

In [0]:
#Loading Trained Model
learn = load_learner(os.getcwd(), 'trained_model.pkl')

In [5]:
path = input("Enter path of test file")

Enter path of test fileapple-fruit.txt


In [0]:
# Adding all paragraphs to a list for testing
test_txt = []
with open(path,"r", encoding="utf8") as f:
  for par in ByParagraph(f):
        if par == "":
            continue
        else:
            test_txt.append(par)

test_df = pd.DataFrame(test_txt, columns = ['text'])

In [0]:
functions = [strip_html_tags, remove_accented_chars, expand_contractions, remove_special_characters]

for fun in functions:
    test_df['text'] = test_df['text'].apply(fun)

In [0]:
preds  = []
for text in test_df['text']:
   preds.append(learn.predict(text)[0])

preds = [str(i) for i in preds]

In [0]:
y_true = ['fruit'] * test_df.shape[0]

In [20]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, preds)

0.9024390243902439